In [23]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
import scipy


from collections import namedtuple

import logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger('bilby')

In [24]:
param_list_short = ['chirp_mass','mass_ratio', 'a_1', 'a_2', 'theta_jn', 'luminosity_distance']
param_labels = [r"$\mathcal{M}_c$", r"$q$", r"$a_1$", r"$a_2$", r"$\theta_{jn}$", r"$d_L$"]


In [25]:
with open("credible_levels.json", "r") as f:
    data = json.load(f)

In [26]:
data.keys()

dict_keys(['chirp_mass', 'mass_ratio', 'a_1', 'a_2', 'theta_jn', 'luminosity_distance'])

In [34]:
x_values = np.linspace(0, 1, 1001)
for process in ["Unclean", "DeepClean"]:

    credible_levels = {
        col: np.asarray(data[col][process])
        for col in param_list_short
    }

    pvalues = []
    logger.info(f"Process: {process}")
    logger.info("Key: KS-test p-value")

    for key, values in credible_levels.items():
        # PP-plot empirical CDF
        pp = np.array([
            np.sum(values < xx) / len(values)
            for xx in x_values
        ])

        # KS-test vs uniform
        pvalue = scipy.stats.kstest(values, 'uniform').pvalue
        pvalues.append(pvalue)
        logger.info(f"  {key}: {pvalue:.3g}")

    # combine p-value 
    combined_p = scipy.stats.combine_pvalues(pvalues)[1]
    logger.info(f"  {'Combined p-value'}: {combined_p:.3g}")

    # store results neatly in a namedtuple
    Pvals = namedtuple('pvals', ['combined_pvalue', 'pvalues', 'names'])
    pvals = Pvals(
        combined_pvalue=combined_p,
        pvalues=pvalues,
        names=list(credible_levels.keys())
    )

    logger.info(f"{'='*40}\n")

INFO: Process: Unclean
INFO: Key: KS-test p-value
INFO:   chirp_mass: 0.361
INFO:   mass_ratio: 0.0275
INFO:   a_1: 0.00213
INFO:   a_2: 0.00115
INFO:   theta_jn: 5.45e-07
INFO:   luminosity_distance: 1.46e-06
INFO:   Combined p-value: 3.47e-14
INFO: ========================================

INFO: Process: DeepClean
INFO: Key: KS-test p-value
INFO:   chirp_mass: 0.54
INFO:   mass_ratio: 0.679
INFO:   a_1: 0.265
INFO:   a_2: 0.312
INFO:   theta_jn: 0.0371
INFO:   luminosity_distance: 0.00585
INFO:   Combined p-value: 0.0212
INFO: ========================================



In [22]:
# for k, col in enumerate(param_list_short):
#     fig, ax = plt.subplots()

#     # Histogrammes PDF
#     ax.hist(
#         [data[col]["Unclean"], data[col]["DeepClean"]],
#         label=['Unclean', 'DeepClean'],
#         histtype='step',
#         density=1,
#         bins=50)
#     ax.set_ylim(0, None)
#     ax.set_ylabel('pdf')
    
#     # CDF
#     ax2 = ax.twinx()
#     ax2.set_ylabel('cdf')
#     ax2.set_ylim(0, 1)
#     ax2.plot(np.sort(data[col]["Unclean"]), np.linspace(0, 1, len(data[col]["Unclean"])), label='Unclean')
#     ax2.plot(np.sort(data[col]["DeepClean"]), np.linspace(0, 1, len(data[col]["DeepClean"])), label='DeepClean')
    
#     # Legend
#     ax.legend(loc='lower right')
#     ax2.legend(loc='upper right')

#     # KS test
#     stat, pvalue = ks_2samp(data[col]["Unclean"], data[col]["DeepClean"])
#     ax.set_xlabel(param_labels[k])
#     ax.set_title(f'K-S test statistic={stat:0.3f}, P-value={pvalue:0.3g}')
#     fig.savefig(f'ks_{col}.png', dpi=300)